##### <p> Samuel Wolfe <br> July 23rd, 2023 <br> MSBA 206 <br> DMBA Chapter 8 Homework 8.1 & 8.2 </p>

In [56]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt
import numpy as np

In [57]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats

# 8.1

#### Loading df
#### Note: With Python 3.11.4 pd.get_dummies() does not default return values as uint8, returns as bool. Have to change post.

In [58]:
dfBank = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/UniversalBank.csv")
dfBank['Education'] = dfBank['Education'].astype('category')
dfBank.columns = dfBank.columns.str.replace(' ', '_')
dfBank = pd.get_dummies(dfBank, prefix_sep='_', drop_first=False)
dfBank['Education_1'] = dfBank['Education_1'].astype('uint8')
dfBank['Education_2'] = dfBank['Education_2'].astype('uint8')
dfBank['Education_3'] = dfBank['Education_3'].astype('uint8')
print(dfBank.shape)
dfBank.head(5)

(5000, 16)


,ID,Age,Experience,Income,ZIP_Code,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_1,Education_2,Education_3
0,1,25,1,49,91107,4,1.6,0,0,1,0,0,0,1,0,0
1,2,45,19,34,90089,3,1.5,0,0,1,0,0,0,1,0,0
2,3,39,15,11,94720,1,1.0,0,0,0,0,0,0,1,0,0
3,4,35,9,100,94112,1,2.7,0,0,0,0,0,0,0,1,0
4,5,35,8,45,91330,4,1.0,0,0,0,0,0,1,0,1,0


#### Creating training and validation splits.

In [59]:
bank_trainData, bank_validData = train_test_split(dfBank, test_size=0.4, random_state=1)
print(bank_trainData.shape, bank_validData.shape)

(3000, 16) (2000, 16)


## 8.1.a Pivot Table

In [60]:
bank_trainData_8_1_a = bank_trainData[['CreditCard', 'Personal_Loan','Online']]
bank_short_PT_8_1_a = bank_trainData_8_1_a.pivot_table(index=['CreditCard', 'Personal_Loan'],
                    columns=['Online'], aggfunc=len)
bank_short_PT_8_1_a

Online                      0     1
CreditCard Personal_Loan           
0          0              792  1117
           1               73   126
1          0              327   477
           1               39    49

## 8.1.b Probability of 1,1,1

In [61]:
prob111_S = f'P(Loan = 1 | CC = 1 | Online = 1): {(bank_short_PT_8_1_a.iat[3,1]) / (bank_short_PT_8_1_a.iat[2,1] + bank_short_PT_8_1_a.iat[3,1]):.2%}'
prob111_S

'P(Loan = 1 | CC = 1 | Online = 1): 9.32%'

## 8.1.c

In [62]:
bank_trainData_8_1_c_1 = bank_trainData[['Personal_Loan','Online']]
bank_short_PT_8_1_c_1 = bank_trainData_8_1_c_1.pivot_table(index=['Personal_Loan'],
                    columns=['Online'], aggfunc=len, margins=True)
bank_short_PT_8_1_c_1

Online,0,1,All
Personal_Loan,,,
0,1119,1594,2713
1,112,175,287
All,1231,1769,3000


In [63]:
bank_trainData_8_1_c_2 = bank_trainData[['CreditCard', 'Personal_Loan']]
bank_short_PT_8_1_c_2 = bank_trainData_8_1_c_2.pivot_table(index=['Personal_Loan'],
                    columns=['CreditCard'], aggfunc=len, margins=True)
bank_short_PT_8_1_c_2

CreditCard,0,1,All
Personal_Loan,,,
0,1909,804,2713
1,199,88,287
All,2108,892,3000


#### 8.1.d.i

In [64]:
P_CC1_Loan1_S = f'P(CC = 1 ∣ Loan = 1): {(bank_short_PT_8_1_c_2.iat[1,1]) / (bank_short_PT_8_1_c_2.iat[1,2]):.2%}'
P_CC1_Loan1 = (bank_short_PT_8_1_c_2.iat[1,1]) / (bank_short_PT_8_1_c_2.iat[1,2])
P_CC1_Loan1_S

'P(CC = 1 ∣ Loan = 1): 30.66%'

#### 8.1.d.ii

In [65]:
P_Online1_Loan1_S = f'P(Online = 1 ∣ Loan = 1): {(bank_short_PT_8_1_c_1.iat[1,1]) / (bank_short_PT_8_1_c_1.iat[2,1]):.2%}'
P_Online1_Loan1 = (bank_short_PT_8_1_c_1.iat[1,1]) / (bank_short_PT_8_1_c_1.iat[2,1])
P_Online1_Loan1_S

'P(Online = 1 ∣ Loan = 1): 9.89%'

#### 8.1.d.iii

In [66]:
P_Loan1_LoanAll_S = f'P(Loan = 1) (the proportion of loan acceptors): {(bank_short_PT_8_1_c_2.iat[1,2]) / (bank_short_PT_8_1_c_2.iat[2,2]):.2%}'
P_Loan1_LoanAll = (bank_short_PT_8_1_c_2.iat[1,2]) / (bank_short_PT_8_1_c_2.iat[2,2])
P_Loan1_LoanAll_S

'P(Loan = 1) (the proportion of loan acceptors): 9.57%'

#### 8.1.d.iv

In [67]:
P_CC1_Loan0_S = f'P(CC = 1 ∣ Loan = 0): {(bank_short_PT_8_1_c_2.iat[0,1]) / (bank_short_PT_8_1_c_2.iat[0,2]):.2%}'
P_CC1_Loan0 = (bank_short_PT_8_1_c_2.iat[0,1]) / (bank_short_PT_8_1_c_2.iat[0,2])
P_CC1_Loan0_S

'P(CC = 1 ∣ Loan = 0): 29.64%'

#### 8.1.d.v

In [68]:
P_Online1_Loan0_S = f'P(Online = 1 ∣ Loan = 0): {(bank_short_PT_8_1_c_1.iat[0,1]) / (bank_short_PT_8_1_c_1.iat[0,2]):.2%}'
P_Online1_Loan0 = (bank_short_PT_8_1_c_1.iat[0,1]) / (bank_short_PT_8_1_c_1.iat[0,2])
P_Online1_Loan0_S

'P(Online = 1 ∣ Loan = 0): 58.75%'

#### 8.1.d.vi

In [69]:
P_Loan0_S = f'P(Loan = 0): {(bank_short_PT_8_1_c_2.iat[0,2]) / (bank_short_PT_8_1_c_2.iat[2,2]):.2%}'
P_Loan0 = (bank_short_PT_8_1_c_2.iat[0,2]) / (bank_short_PT_8_1_c_2.iat[2,2])
P_Loan0_S

'P(Loan = 0): 90.43%'

## 8.1.e

In [70]:
P_Bayes_Loan1_CC1_Online1 = f'Naive Bayes P(Loan = 1 ∣ CC = 1, Online = 1): {(P_CC1_Loan1 * P_Online1_Loan1 * P_Loan1_LoanAll)/ ((P_CC1_Loan1 * P_Online1_Loan1 * P_Loan1_LoanAll) + (P_CC1_Loan0 * P_Online1_Loan0 * P_Loan0)):.2%}'
P_Bayes_Loan1_CC1_Online1

'Naive Bayes P(Loan = 1 ∣ CC = 1, Online = 1): 1.81%'

## 8.1.f 
#### Looking at the raw values, the value generated from the first pivot table in 8.1.b is higher at 9.32% while in 8.1.e the naive Bayes probability is 1.81%. Knowing that Naive Bayes makes the assumption that all variables are independent (hence naive) I would say the first pivot table solution in 8.1.b is more accurate as it takes into consideration all the values.

## 8.1.e

#### Preparing data

In [71]:
classes = [0,1]
predictors = ['Online','CreditCard']
outcome = ['Personal_Loan']

dfBank = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/UniversalBank.csv")
dfBank['Education'] = dfBank['Education'].astype('category')
dfBank.columns = dfBank.columns.str.replace(' ', '_')
X_dfBank = pd.get_dummies(dfBank[predictors], prefix_sep='_', drop_first=False)
Y_dfBank = (dfBank[outcome] == 'Personal_Loan').astype(int)


#scaler = MinMaxScaler()
#scaler.fit(X_dfBank)
#X_dfBank = pd.DataFrame(scaler.transform(X_dfBank),index=X_dfBank.index, columns=X_dfBank.columns)

X_train, X_valid, y_train, y_valid = train_test_split(X_dfBank, Y_dfBank, test_size=0.40, random_state=1)

#### Run Naive Bayes

In [72]:
bank_nb = MultinomialNB(alpha=0.01)
bank_nb.fit(X_train, y_train.values.ravel())

# predict probabilities
predProb_train = bank_nb.predict_proba(X_train)
predProb_valid = bank_nb.predict_proba(X_valid)

# predict class membership
y_train_pred = bank_nb.predict(X_train)
y_valid_pred = bank_nb.predict(X_valid)

In [73]:
predProb_valid

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [74]:
y_valid_pred

array([0, 0, 0, ..., 0, 0, 0])

In [75]:
train_df, valid_df = train_test_split(dfBank, test_size=0.4, random_state=1)

pd.set_option('display.precision', 4)
# probability of flight status
print(train_df['Personal_Loan'].value_counts() / len(train_df))
print()

for predictor in predictors:
    # construct the frequency table
    df = train_df[['Personal_Loan', predictor]]
    freqTable = df.pivot_table(index='Personal_Loan', columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')

Personal_Loan
0    0.9043
1    0.0957
Name: count, dtype: float64

Online              0       1
Personal_Loan                
0              0.4125  0.5875
1              0.3902  0.6098

CreditCard          0       1
Personal_Loan                
0              0.7036  0.2964
1              0.6934  0.3066



In [78]:
# classify a specific flight by searching in the dataset
# for a flight with the same predictor values
df = pd.concat([pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred}),
                pd.DataFrame(predProb_valid, index=y_valid.index)], axis=1)
mask = ((bank_validData.Online == 1) & (bank_validData.CreditCard == 1) &
        (bank_validData.Personal_Loan == 1))
df[mask]

ValueError: Data must be 1-dimensional, got ndarray of shape (2000, 1) instead

In [ ]:
# training
classificationSummary(bank_trainY, bank_pred_train, class_names=classes)
# validation
classificationSummary(bank_validY, bank_pred_valid, class_names=classes)